### Load Example Scene Graph 

In [ ]:
import jax.numpy as jnp
import bayes3d as b
import numpy as np
import bayes3d.utils.ycb_loader
import trimesh
import jax
import os
from tqdm import tqdm


def test_ycb_loading():
    bop_ycb_dir = os.path.join(b.utils.get_assets_dir(), "bop/ycbv")
    rgbd, gt_ids, gt_poses, masks = b.utils.ycb_loader.get_test_img('52', '1', bop_ycb_dir)

    b.setup_renderer(rgbd.intrinsics, num_layers=1)

    model_dir =os.path.join(b.utils.get_assets_dir(), "bop/ycbv/models")
    for idx in range(1,22):
        b.RENDERER.add_mesh_from_file(os.path.join(model_dir,"obj_" + "{}".format(idx).rjust(6, '0') + ".ply"),scaling_factor=1.0/1000.0)

    reconstruction_depth = b.RENDERER.render(gt_poses, gt_ids)[:,:,2]
    match_fraction = (jnp.abs(rgbd.depth - reconstruction_depth) < 0.05).mean()
    assert match_fraction > 0.2

bop_ycb_dir = os.path.join(b.utils.get_assets_dir(), "bop/ycbv")
rgbd, gt_ids, gt_poses, masks = b.utils.ycb_loader.get_test_img('55', '22', bop_ycb_dir)
poses = jnp.concatenate([jnp.eye(4)[None,...], rgbd.camera_pose @ gt_poses],axis=0)
ids = jnp.concatenate([jnp.array([21]), gt_ids],axis=0)


b.setup_renderer(rgbd.intrinsics, num_layers=1)

model_dir =os.path.join(b.utils.get_assets_dir(), "bop/ycbv/models")
for idx in range(1,22):
    b.RENDERER.add_mesh_from_file(os.path.join(model_dir,"obj_" + "{}".format(idx).rjust(6, '0') + ".ply"),scaling_factor=1.0/1000.0)

b.RENDERER.add_mesh_from_file(os.path.join(b.utils.get_assets_dir(), "sample_objs/cube.obj"), scaling_factor=1.0/1000000000.0)

scene_graph = b.scene_graph.SceneGraph(
    root_poses=poses,
    box_dimensions=b.RENDERER.model_box_dims[ids],
    parents=jnp.full(poses.shape[0], -1),
    contact_params=jnp.zeros((poses.shape[0],3)),
    face_parent=jnp.zeros(poses.shape[0], dtype=jnp.int32),
    face_child=jnp.zeros(poses.shape[0], dtype=jnp.int32),
)
assert jnp.isclose(scene_graph.get_poses(), poses).all()

def get_slack(scene_graph, parent_object_index, child_object_index, face_parent, face_child):
    parent_pose = scene_graph.get_poses()[parent_object_index]
    child_pose = scene_graph.get_poses()[child_object_index]
    dims_parent = scene_graph.box_dimensions[parent_object_index]
    dims_child = scene_graph.box_dimensions[child_object_index]
    parent_contact_plane = parent_pose @ b.scene_graph.get_contact_planes(dims_parent)[face_parent]
    child_contact_plane = child_pose @ b.scene_graph.get_contact_planes(dims_child)[face_child]

    contact_params, slack = b.scene_graph.closest_approximate_contact_params(parent_contact_plane, child_contact_plane)
    return jnp.array([parent_object_index, child_object_index, face_parent, face_child]), contact_params, slack

add_edge_scene_graph = jax.jit(b.scene_graph.add_edge_scene_graph)

N = poses.shape[0]
# b.setup_visualizer()

get_slack_vmap = jax.jit(b.utils.multivmap(get_slack, (False, False, False, True, True)))

edges = [(0,1),(0,2),(0,3),(0,4),(0,6),(2,5)]
for i,j in edges:
    settings, contact_params, slacks = get_slack_vmap(scene_graph, i,j, jnp.arange(6), jnp.arange(6))
    settings = settings.reshape(-1,settings.shape[-1])
    contact_params = contact_params.reshape(-1,contact_params.shape[-1])
    error = jnp.abs(slacks - jnp.eye(4)).sum([-1,-2]).reshape(-1)
    indices = jnp.argsort(error.reshape(-1))

    parent_object_index, child_object_index, face_parent, face_child = settings[indices[0]]
    scene_graph = add_edge_scene_graph(scene_graph,parent_object_index, child_object_index, face_parent, face_child, contact_params[indices[0]])

node_names = np.array([*b.utils.ycb_loader.MODEL_NAMES, "table"])
scene_graph.table_visualize("graph.png", node_names=list(map(str,enumerate(node_names[ids]))))

In [ ]:
from PIL import Image as im 
# Show YCB Image
np_rep = np.array(rgbd.rgb.copy())
image  = im.fromarray(np_rep)
image.save("base_ycb/55_22.png")
scene_graph.table_visualize("scene_graphs/55_22.svg", node_names=list(map(str,enumerate(node_names[ids]))))
image

In [ ]:
# Show Scene Graph 
graph_vis = im.open("graph.png")
graph_vis

In [ ]:
from bayes3d._rendering.photorealistic_renderers.kubric_interface import render_many
# create mesh_paths, could take from scene_graph construction. 
model_dir =os.path.join(b.utils.get_assets_dir(), "ycb_video_models/models")
model_paths = []
for model_name in node_names[ids]:
    model_paths.append(os.path.join(model_dir,model_name +"/textured.obj"))

# add table as root node 
model_paths[0] = os.path.join(b.utils.get_assets_dir(), "sample_objs/table.obj")
poses = scene_graph.get_poses()
intrinsics = rgbd.intrinsics
scaling_factor = 1.0

# Pass through and render 
outputs = render_many(model_paths, poses[None,...], intrinsics, scaling_factor=scaling_factor, camera_pose = rgbd.camera_pose)

In [ ]:
poses.shape

In [ ]:
image = b.get_rgb_image(outputs[0].rgb)
image.save(f"reconstruction/{55}_{22}.png")

## Repeat for a few scenes

In [ ]:
import jax.numpy as jnp
import bayes3d as b
import numpy as np
import bayes3d.utils.ycb_loader
import trimesh
import jax
import os
from tqdm import tqdm
from bayes3d._rendering.photorealistic_renderers.kubric_interface import render_many
from PIL import Image as im 


def ycb_load(test_set, image_number):
    test_set = str(test_set)
    image_number = str(image_number)
    bop_ycb_dir = os.path.join(b.utils.get_assets_dir(), "bop/ycbv")
    rgbd, gt_ids, gt_poses, masks = b.utils.ycb_loader.get_test_img(test_set, image_number, bop_ycb_dir)
    poses = jnp.concatenate([jnp.eye(4)[None,...], rgbd.camera_pose @ gt_poses],axis=0)
    ids = jnp.concatenate([jnp.array([21]), gt_ids],axis=0)


    b.setup_renderer(rgbd.intrinsics, num_layers=1) 

    model_dir =os.path.join(b.utils.get_assets_dir(), "bop/ycbv/models")
    for idx in range(1,22):
        b.RENDERER.add_mesh_from_file(os.path.join(model_dir,"obj_" + "{}".format(idx).rjust(6, '0') + ".ply"),scaling_factor=1.0/1000.0)

    b.RENDERER.add_mesh_from_file(os.path.join(b.utils.get_assets_dir(), "sample_objs/cube.obj"), scaling_factor=1.0/1000000000.0)

    scene_graph = b.scene_graph.SceneGraph(
        root_poses=poses,
        box_dimensions=b.RENDERER.model_box_dims[ids],
        parents=jnp.full(poses.shape[0], -1),
        contact_params=jnp.zeros((poses.shape[0],3)),
        face_parent=jnp.zeros(poses.shape[0], dtype=jnp.int32),
        face_child=jnp.zeros(poses.shape[0], dtype=jnp.int32),
    )
    assert jnp.isclose(scene_graph.get_poses(), poses).all()

    # add_edge_scene_graph = jax.jit(b.scene_graph.add_edge_scene_graph)

    N = poses.shape[0]

    get_slack_vmap = jax.jit(b.utils.multivmap(get_slack, (False, False, False, True, True)))
    return scene_graph, rgbd, ids

def get_slack(scene_graph, parent_object_index, child_object_index, face_parent, face_child):
    parent_pose = scene_graph.get_poses()[parent_object_index]
    child_pose = scene_graph.get_poses()[child_object_index]
    dims_parent = scene_graph.box_dimensions[parent_object_index]
    dims_child = scene_graph.box_dimensions[child_object_index]
    parent_contact_plane = parent_pose @ b.scene_graph.get_contact_planes(dims_parent)[face_parent]
    child_contact_plane = child_pose @ b.scene_graph.get_contact_planes(dims_child)[face_child]

    contact_params, slack = b.scene_graph.closest_approximate_contact_params(parent_contact_plane, child_contact_plane)
    return jnp.array([parent_object_index, child_object_index, face_parent, face_child]), contact_params, slack
add_edge_scene_graph = jax.jit(b.scene_graph.add_edge_scene_graph)
get_slack_vmap = jax.jit(b.utils.multivmap(get_slack, (False, False, False, True, True)))

def vis_scene_graph(scene_graph,edges, name = "graph.png"): 
    for i,j in edges:
        settings, contact_params, slacks = get_slack_vmap(scene_graph, i,j, jnp.arange(6), jnp.arange(6))
        settings = settings.reshape(-1,settings.shape[-1])
        contact_params = contact_params.reshape(-1,contact_params.shape[-1])
        error = jnp.abs(slacks - jnp.eye(4)).sum([-1,-2]).reshape(-1)
        indices = jnp.argsort(error.reshape(-1))

        parent_object_index, child_object_index, face_parent, face_child = settings[indices[0]]
        scene_graph = add_edge_scene_graph(scene_graph,parent_object_index, child_object_index, face_parent, face_child, contact_params[indices[0]])

    node_names = np.array([*b.utils.ycb_loader.MODEL_NAMES, "table"])
    scene_graph.table_visualize(name, node_names=list(map(str,enumerate(node_names[ids]))))
    return node_names 

def render_sg_kubric(node_names, scene_graph, rgbd):
    # create mesh_paths, could take from scene_graph construction. 
    model_dir =os.path.join(b.utils.get_assets_dir(), "ycb_video_models/models")
    model_paths = []
    for model_name in node_names[ids]:
        model_paths.append(os.path.join(model_dir,model_name +"/textured.obj"))

    # add table as root node 
    model_paths[0] = os.path.join(b.utils.get_assets_dir(), "sample_objs/plane.obj")
    poses = scene_graph.get_poses()
    intrinsics = rgbd.intrinsics
    scaling_factor = 1.0

    # Pass through and render (note; it seems the ycb requires a rotation)
    camera_pose = rgbd.camera_pose @ b.t3d.transform_from_axis_angle(jnp.array([1.0, 0.0,0.0]), jnp.pi)
    outputs = render_many(model_paths, poses[None,...], intrinsics, scaling_factor=scaling_factor, camera_pose = rgbd.camera_pose)
    return outputs

### 49-1 

In [ ]:
set_num, img_num = 49,1
scene_graph, rgbd, ids = ycb_load(set_num,img_num)
np_rep = np.array(rgbd.rgb.copy())
image  = im.fromarray(np_rep)
file = f"{set_num}_{img_num}"
image.save(f"base_ycb/{file}.png")
image

In [ ]:
edges = [(0,2), (0,4), (0,3), (4,1)]

name = f"{file}.svg"
node_names = vis_scene_graph(scene_graph, edges,name)
print(node_names)
graph_vis = im.open(name)
graph_vis

In [ ]:
outputs = render_sg_kubric(node_names, scene_graph, rgbd)

In [ ]:
image = b.get_rgb_image(outputs[0].rgb)
image.save(f"reconstruction/{file}.png")
image

### 51-1 

In [ ]:
scene_graph, rgbd, ids = ycb_load(51,1)
np_rep = np.array(rgbd.rgb.copy())
image  = im.fromarray(np_rep)
image.save(f"base_ycb/{51}_{1}.png")
image

In [ ]:
edges = [(0,1), (0,3),(0,4),(0,5),(5,2)]

name = "scene_graphs/51_1.svg"
node_names = vis_scene_graph(scene_graph, edges,name)
print(node_names[ids])
graph_vis = im.open(name)
graph_vis

In [ ]:
outputs = render_sg_kubric(node_names, scene_graph, rgbd)
image = b.get_rgb_image(outputs[0].rgb)
image.save(f"reconstruction/{51}_{1}.png")
image

### 50-620

In [ ]:
set_num, img_num = 50,620
scene_graph, rgbd, ids = ycb_load(set_num,img_num)
np_rep = np.array(rgbd.rgb.copy())
image  = im.fromarray(np_rep)
file = f"{set_num}_{img_num}"
image.save(f"base_ycb/{file}.png")
image

In [ ]:
edges = [(0,2), (0,4), (0,3), (0,1), (0,5)]

name = f"scene_graphs/{file}.svg"
node_names = vis_scene_graph(scene_graph, edges,name)
print(node_names)
# graph_vis = im.open(name)
# graph_vis

In [ ]:
outputs = render_sg_kubric(node_names, scene_graph, rgbd)
image = b.get_rgb_image(outputs[0].rgb)
image.save(f"reconstruction/{file}.png")
image

### 54-1

In [ ]:
set_num, img_num = 54,1
scene_graph, rgbd, ids = ycb_load(set_num,img_num)
np_rep = np.array(rgbd.rgb.copy())
image  = im.fromarray(np_rep)
file = f"{set_num}_{img_num}"
image.save(f"base_ycb/{file}.png")
image

In [ ]:
edges = [(0,2), (0,4), (0,3), (0,1), (0,5)]

name = f"scene_graphs/{file}.svg"
node_names = vis_scene_graph(scene_graph, edges,name)
print(node_names)

In [ ]:
outputs = render_sg_kubric(node_names, scene_graph, rgbd)
image = b.get_rgb_image(outputs[0].rgb)
image.save(f"reconstruction/{file}.png")
image

### 56-1

In [ ]:
set_num, img_num = 56,1
scene_graph, rgbd, ids = ycb_load(set_num,img_num)
np_rep = np.array(rgbd.rgb.copy())
image  = im.fromarray(np_rep)
file = f"{set_num}_{img_num}"
image.save(f"base_ycb/{file}.png")
image

In [ ]:
edges = [(0,2), (0,4), (0,3), (0,1), (0,5)]

name = f"scene_graphs/{file}.svg"
node_names = vis_scene_graph(scene_graph, edges,name)
print(node_names)

In [ ]:
outputs = render_sg_kubric(node_names, scene_graph, rgbd)
image = b.get_rgb_image(outputs[0].rgb)
image.save(f"reconstruction/{file}.png")
image

### 57-1

In [ ]:
set_num, img_num = 57,1
scene_graph, rgbd, ids = ycb_load(set_num,img_num)
np_rep = np.array(rgbd.rgb.copy())
image  = im.fromarray(np_rep)
file = f"{set_num}_{img_num}"
image.save(f"base_ycb/{file}.png")
image

In [ ]:
edges = [(0,2), (0,4), (5,3), (0,1), (0,5)]

name = f"scene_graphs/{file}.svg"
node_names = vis_scene_graph(scene_graph, edges,name)
print(node_names[ids])

In [ ]:
outputs = render_sg_kubric(node_names, scene_graph, rgbd)
image = b.get_rgb_image(outputs[0].rgb)
image.save(f"reconstruction/{file}.png")
image

### 58-30

In [ ]:
set_num, img_num = 58,30
scene_graph, rgbd, ids = ycb_load(set_num,img_num)
np_rep = np.array(rgbd.rgb.copy())
image  = im.fromarray(np_rep)
file = f"{set_num}_{img_num}"
image.save(f"base_ycb/{file}.png")
image

In [ ]:
edges = [(0,2), (0,4), (0,3), (0,1), (0,5)]

name = f"scene_graphs/{file}.svg"
node_names = vis_scene_graph(scene_graph, edges,name)
print(node_names)

In [ ]:
outputs = render_sg_kubric(node_names, scene_graph, rgbd)
image = b.get_rgb_image(outputs[0].rgb)
image.save(f"reconstruction/{file}.png")
image

### 59-1

In [ ]:
set_num, img_num = 59,1
scene_graph, rgbd, ids = ycb_load(set_num,img_num)
np_rep = np.array(rgbd.rgb.copy())
image  = im.fromarray(np_rep)
file = f"{set_num}_{img_num}"
image.save(f"base_ycb/{file}.png")
image

In [ ]:
edges = [(0,2), (0,4), (0,3), (0,1), (0,5), (0,6)]

name = f"scene_graphs/{file}.svg"
node_names = vis_scene_graph(scene_graph, edges,name)
print(node_names)

In [ ]:
outputs = render_sg_kubric(node_names, scene_graph, rgbd)
image = b.get_rgb_image(outputs[0].rgb)
image.save(f"reconstruction/{file}.png")
image

### 48-1

In [ ]:
set_num, img_num = 48,1
scene_graph, rgbd, ids = ycb_load(set_num,img_num)
np_rep = np.array(rgbd.rgb.copy())
image  = im.fromarray(np_rep)
file = f"{set_num}_{img_num}"
image.save(f"base_ycb/{file}.png")
image

In [ ]:
edges = [(0,2), (0,4), (1,3), (0,1), (0,5)]

name = f"scene_graphs/{file}.svg"
node_names = vis_scene_graph(scene_graph, edges,name)
print(node_names[ids])

In [ ]:
outputs = render_sg_kubric(node_names, scene_graph, rgbd)
image = b.get_rgb_image(outputs[0].rgb)
image.save(f"reconstruction/{file}.png")
image

## Add to pbs Scene

In [ ]:
import jax.numpy as jnp
import bayes3d as b
import numpy as np
import bayes3d.utils.ycb_loader
import trimesh
import jax
from bayes3d.utils import pybullet_sim as pbs
import os
from tqdm import tqdm
from bayes3d._rendering.photorealistic_renderers.kubric_interface import render_many
from PIL import Image as im 

scenes = [[49,1],[51,1], [50,620], [54,1], [56,1], [57,1], [58,30], [59,1], [48,1]]

def ycb_load(test_set, image_number):
    test_set = str(test_set)
    image_number = str(image_number)
    bop_ycb_dir = os.path.join(b.utils.get_assets_dir(), "bop/ycbv")
    rgbd, gt_ids, gt_poses, masks = b.utils.ycb_loader.get_test_img(test_set, image_number, bop_ycb_dir)
    poses = jnp.concatenate([jnp.eye(4)[None,...], rgbd.camera_pose @ gt_poses],axis=0)
    ids = jnp.concatenate([jnp.array([21]), gt_ids],axis=0)


    b.setup_renderer(rgbd.intrinsics, num_layers=1) 

    model_dir =os.path.join(b.utils.get_assets_dir(), "bop/ycbv/models")
    for idx in range(1,22):
        b.RENDERER.add_mesh_from_file(os.path.join(model_dir,"obj_" + "{}".format(idx).rjust(6, '0') + ".ply"),scaling_factor=1.0/1000.0)

    b.RENDERER.add_mesh_from_file(os.path.join(b.utils.get_assets_dir(), "sample_objs/cube.obj"), scaling_factor=1.0/1000000000.0)

    scene_graph = b.scene_graph.SceneGraph(
        root_poses=poses,
        box_dimensions=b.RENDERER.model_box_dims[ids],
        parents=jnp.full(poses.shape[0], -1),
        contact_params=jnp.zeros((poses.shape[0],3)),
        face_parent=jnp.zeros(poses.shape[0], dtype=jnp.int32),
        face_child=jnp.zeros(poses.shape[0], dtype=jnp.int32),
    )
    assert jnp.isclose(scene_graph.get_poses(), poses).all()

    # add_edge_scene_graph = jax.jit(b.scene_graph.add_edge_scene_graph)

    N = poses.shape[0]

    get_slack_vmap = jax.jit(b.utils.multivmap(get_slack, (False, False, False, True, True)))
    return scene_graph, rgbd, ids

def get_slack(scene_graph, parent_object_index, child_object_index, face_parent, face_child):
    parent_pose = scene_graph.get_poses()[parent_object_index]
    child_pose = scene_graph.get_poses()[child_object_index]
    dims_parent = scene_graph.box_dimensions[parent_object_index]
    dims_child = scene_graph.box_dimensions[child_object_index]
    parent_contact_plane = parent_pose @ b.scene_graph.get_contact_planes(dims_parent)[face_parent]
    child_contact_plane = child_pose @ b.scene_graph.get_contact_planes(dims_child)[face_child]

    contact_params, slack = b.scene_graph.closest_approximate_contact_params(parent_contact_plane, child_contact_plane)
    return jnp.array([parent_object_index, child_object_index, face_parent, face_child]), contact_params, slack
add_edge_scene_graph = jax.jit(b.scene_graph.add_edge_scene_graph)
get_slack_vmap = jax.jit(b.utils.multivmap(get_slack, (False, False, False, True, True)))

def vis_scene_graph(scene_graph, edges, name = "graph.png"): 
    for i,j in edges:
        settings, contact_params, slacks = get_slack_vmap(scene_graph, i,j, jnp.arange(6), jnp.arange(6))
        settings = settings.reshape(-1,settings.shape[-1])
        contact_params = contact_params.reshape(-1,contact_params.shape[-1])
        error = jnp.abs(slacks - jnp.eye(4)).sum([-1,-2]).reshape(-1)
        indices = jnp.argsort(error.reshape(-1))

        parent_object_index, child_object_index, face_parent, face_child = settings[indices[0]]
        scene_graph = add_edge_scene_graph(scene_graph,parent_object_index, child_object_index, face_parent, face_child, contact_params[indices[0]])

    node_names = np.array([*b.utils.ycb_loader.MODEL_NAMES, "table"])
    scene_graph.table_visualize(name, node_names=list(map(str,enumerate(node_names[ids]))))
    return node_names 

def render_sg_kubric(node_names, scene_graph, rgbd):
    # create mesh_paths, could take from scene_graph construction. 
    model_dir =os.path.join(b.utils.get_assets_dir(), "ycb_video_models/models")
    model_paths = []
    for model_name in node_names[ids]:
        model_paths.append(os.path.join(model_dir,model_name +"/textured.obj"))

    # add table as root node 
    model_paths[0] = os.path.join(b.utils.get_assets_dir(), "sample_objs/plane.obj")
    poses = scene_graph.get_poses()
    intrinsics = rgbd.intrinsics
    scaling_factor = 1.0

    # Pass through and render 
    outputs = render_many(model_paths, poses[None,...], intrinsics, scaling_factor=scaling_factor, camera_pose = rgbd.camera_pose)
    return outputs

def model_paths(node_names, scene_graph, rgbd): 
    model_dir =os.path.join(b.utils.get_assets_dir(), "ycb_video_models/models")
    model_paths = []
    for model_name in node_names[ids]:
        model_paths.append(os.path.join(model_dir,model_name +"/textured.obj"))

    # add table as root node 
    model_paths[0] = os.path.join(b.utils.get_assets_dir(), "sample_objs/plane.obj")
    poses = scene_graph.get_poses()
    intrinsics = rgbd.intrinsics
    scaling_factor = 1.0
    scene = pbs.Scene() 

In [ ]:
for set_num, img_num in scenes: 
    scene_graph, rgbd, ids = ycb_load(set_num,img_num)
    edges = [] 
    node_names = vis_scene_graph(scene_graph, edges)

    model_dir =os.path.join(b.utils.get_assets_dir(), "ycb_video_models/models")
    model_paths = []
    for model_name in node_names[ids]:
        model_paths.append(os.path.join(model_dir,model_name +"/textured.obj"))

    # add table as root node 
    model_paths[0] = os.path.join(b.utils.get_assets_dir(), "sample_objs/plane.obj")
    poses = scene_graph.get_poses()
    intrinsics = rgbd.intrinsics

    scene = pbs.Scene(floor=True)
    scene.set_camera_pose(rgbd.camera_pose)
    scene.camera.set_intrinsics(intrinsics)
    scene.set_gravity([0,0,-1])

    body_names = node_names[ids]
    for i, name in enumerate(body_names): 
        if i != 0: 
            body_pose = poses[i]
            mesh = model_paths[i]
            body = pbs.make_body_from_obj_pose(mesh, body_pose, id = name)
            body.set_restitution(0)
            scene.add_body(body)

    path = "../assets/sample_objs/sphere.obj"
    position = [0,-1,.1]
    velocity = [0,4,0]
    scale = np.array([1,1,1]) * .1 
    bowling = pbs.make_body_from_obj(path, position, scale = scale,id = "bowling")
    bowling.set_velocity(velocity)
    bowling.set_restitution(0)
    bowling.set_color(np.array([.1,.1,.1]))
    bowling.set_mass = 10000
    scene.add_body(bowling)

    pyb = scene.simulate(30)
    pyb.create_gif(f"ycb_pyb/{set_num}_{img_num}.gif", 30)